In [1]:
%cd ../../../../

/home/users/dmoreno2016/ASTROMER/astromer_pe/astromer


In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import copy

import toml
import os

from src.models.astromer_1 import get_ASTROMER
from src.data.record import deserialize

%load_ext autoreload
%autoreload 2

2023-10-04 01:56:07.664429: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-04 01:56:08.537270: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
def get_data(dataset):
    list_data_lcs = []

    for lc_info in dataset:
        np_lc = lc_info['input'].numpy()
        np_lc = np_lc[np_lc[:,0].argsort()]

        data = {'lcid': lc_info['lcid'].numpy(), 
                'lc_data': [np_lc],
                'label': lc_info['label'].numpy()}  

        list_data_lcs.append(pd.DataFrame(data))

        if np_lc.shape[0] < 200:
            print(lc_info['lcid'].numpy())

    list_data_lcs = pd.concat(list_data_lcs)
    return list_data_lcs

In [4]:
path_dataset = './data/records/alcock/fold_0/alcock_20/train'

batch_size = 16
probed = 0.4
random_same = 0.2 
window_size = 200 
nsp_prob = .5
repeat = 1 
sampling = False
shuffle = False
njobs = None
num_cls = None
test_mode = False
off_nsp = True

In [5]:
rec_paths = []
for folder in os.listdir(path_dataset):
    if folder.endswith('.csv'):
        continue
    for x in os.listdir(os.path.join(path_dataset, folder)):
        rec_paths.append(os.path.join(path_dataset, folder, x))

dataset = tf.data.TFRecordDataset(rec_paths)    
dataset = dataset.map(deserialize)

2023-10-04 01:56:09.733367: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [6]:
dataset = dataset.repeat(repeat)

In [7]:
# Realiza correctamente el sampling 
def sample_lc(sample, max_obs, binary=True):
	'''
	Sample a random window of "max_obs" observations from the input sequence
	'''
	if binary:
		input_dict = deserialize(sample)
	else:
		input_dict = sample

	serie_len = tf.shape(input_dict['input'])[0]

	pivot = 0
	if tf.greater(serie_len, max_obs):
		pivot = tf.random.uniform([],
								  minval=0,
								  maxval=serie_len-max_obs+1,
								  dtype=tf.int32)

		input_dict['input'] = tf.slice(input_dict['input'], [pivot,0], [max_obs, -1]) # input_dict['input'][pivot:max_obs, :-1]
	else:
		input_dict['input'] = tf.slice(input_dict['input'], [0,0], [serie_len, -1])

	return input_dict

samples_dataset = dataset.map(lambda x: sample_lc(x,
                                          max_obs=window_size,
                                          binary=False),
                                          num_parallel_calls=tf.data.experimental.AUTOTUNE)

samples_dataset

<_ParallelMapDataset element_spec={'lcid': TensorSpec(shape=(), dtype=tf.string, name=None), 'length': TensorSpec(shape=(), dtype=tf.int32, name=None), 'label': TensorSpec(shape=(), dtype=tf.int32, name=None), 'input': TensorSpec(shape=(None, None), dtype=tf.float32, name=None)}>

In [8]:
for lc_info in dataset:
    np_lc = lc_info['input'].numpy()
    np_lc = np_lc[np_lc[:,0].argsort()]

In [9]:
np_lc.shape[0]

966

In [10]:
def get_window(sequence, length, pivot, max_obs):
	if length-pivot > max_obs:
		sliced = tf.slice(sequence, [pivot, 0], [max_obs, -1])
	else:
		sliced = tf.slice(sequence, [pivot, 0], [-1, -1])
		
	return sliced

max_obs = 200
pivots = tf.tile([max_obs], [tf.cast(np_lc.shape[0]/max_obs, tf.int32)])
pivots

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([200, 200, 200, 200], dtype=int32)>

In [ ]:
def get_window(sequence, length, pivot, split_size):
    if length - pivot > split_size:
        sliced = tf.slice(sequence, [pivot, 0], [split_size, -1])
    else:
        sliced = tf.slice(sequence, [pivot, 0], [-1, -1])
        
    return sliced

pivots = tf.cumsum(split_sizes) # Calcula los pivotes basados en los tamaños de división

splits = tf.map_fn(lambda x: get_window(np_lc, np_lc.shape[0], x, split_sizes[x]), tf.range(len(split_sizes)),
                    infer_shape=False,
                    fn_output_signature=(tf.float32))

In [16]:
def get_window(sequence, length, pivot, max_obs):
    def fn_true():
        return tf.slice(sequence, [pivot, 0], [max_obs, -1])

    def fn_false():
        return tf.slice(sequence, [pivot, 0], [-1, -1])

    return tf.cond(length - pivot > max_obs, fn_true, fn_false)

max_obs = 200
pivots = tf.tile([max_obs], [tf.cast(np_lc.shape[0] // max_obs, tf.int32)])

splits = tf.map_fn(lambda x: get_window(np_lc, np_lc.shape[0], x, max_obs), pivots,
                    infer_shape=False,
                    fn_output_signature=(tf.float32))

splits

<tf.Tensor: shape=(4, 200, 3), dtype=float32, numpy=
array([[[ 4.9162766e+04, -6.8810000e+00,  2.5000000e-02],
        [ 4.9163379e+04, -6.8230000e+00,  1.2000000e-02],
        [ 4.9164379e+04, -6.8839998e+00,  1.5000000e-02],
        ...,
        [ 4.9538746e+04, -6.8470001e+00,  2.2000000e-02],
        [ 4.9540730e+04, -6.8790002e+00,  1.6000001e-02],
        [ 4.9541805e+04, -6.8699999e+00,  8.0000004e-03]],

       [[ 4.9162766e+04, -6.8810000e+00,  2.5000000e-02],
        [ 4.9163379e+04, -6.8230000e+00,  1.2000000e-02],
        [ 4.9164379e+04, -6.8839998e+00,  1.5000000e-02],
        ...,
        [ 4.9538746e+04, -6.8470001e+00,  2.2000000e-02],
        [ 4.9540730e+04, -6.8790002e+00,  1.6000001e-02],
        [ 4.9541805e+04, -6.8699999e+00,  8.0000004e-03]],

       [[ 4.9162766e+04, -6.8810000e+00,  2.5000000e-02],
        [ 4.9163379e+04, -6.8230000e+00,  1.2000000e-02],
        [ 4.9164379e+04, -6.8839998e+00,  1.5000000e-02],
        ...,
        [ 4.9538746e+04, -6.847000

In [68]:
splits[0].numpy()

(200, 3)

In [85]:
splits[2].numpy()

array([[ 4.9162766e+04, -6.8810000e+00,  2.5000000e-02],
       [ 4.9163379e+04, -6.8230000e+00,  1.2000000e-02],
       [ 4.9164379e+04, -6.8839998e+00,  1.5000000e-02],
       [ 4.9169820e+04, -6.8130002e+00,  1.5000000e-02],
       [ 4.9172375e+04, -6.8880000e+00,  2.0000000e-02],
       [ 4.9178773e+04, -6.8490000e+00,  4.1999999e-02],
       [ 4.9179730e+04, -6.8189998e+00,  2.0000000e-02],
       [ 4.9180699e+04, -6.8660002e+00,  1.7999999e-02],
       [ 4.9181684e+04, -6.8779998e+00,  2.0000000e-02],
       [ 4.9182785e+04, -6.9460001e+00,  1.5000000e-02],
       [ 4.9183691e+04, -6.8480000e+00,  3.0999999e-02],
       [ 4.9184664e+04, -6.8880000e+00,  2.2000000e-02],
       [ 4.9185383e+04, -6.8730001e+00,  1.4000000e-02],
       [ 4.9188668e+04, -6.9089999e+00,  1.7000001e-02],
       [ 4.9189676e+04, -6.9260001e+00,  2.0000000e-02],
       [ 4.9194691e+04, -6.8490000e+00,  1.7000001e-02],
       [ 4.9195641e+04, -6.8670001e+00,  1.6000001e-02],
       [ 4.9196676e+04, -6.9390

In [86]:
splits[2].numpy() == np_lc[200:400]

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ T

In [55]:
966-400 > 200

True

In [56]:
pivots = tf.concat([[0], pivots], 0)
pivots

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([  0, 200, 200, 200, 200], dtype=int32)>

In [57]:
pivots = tf.math.cumsum(pivots)
pivots

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([  0, 200, 400, 600, 800], dtype=int32)>

In [58]:
splits = tf.map_fn(lambda x: get_window(np_lc, np_lc.shape[0], x, max_obs), pivots,
										infer_shape=False,
										fn_output_signature=(tf.float32))
splits

InvalidArgumentError: in user code:

    File "/tmp/ipykernel_4524/1436799668.py", line 1, in None  *
        lambda x: get_window(np_lc, np_lc.shape[0], x, max_obs)
    File "/tmp/ipykernel_4524/535930636.py", line 9, in get_window  *
        sliced = tf.slice(sequence, [start, 0], [end - start, -1])

    InvalidArgumentError: {{function_node __wrapped__Slice_device_/job:localhost/replica:0/task:0/device:CPU:0}} Expected size[0] in [0, 166], but got 200 [Op:Slice]


In [ ]:
np_lc[0:200] == splits[0].numpy()

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ T

In [ ]:
def get_window(sequence, length, pivot, max_obs):
	pivot = tf.minimum(length-max_obs, pivot)
	pivot = tf.maximum(0, pivot)
	end = tf.minimum(length, max_obs)

	sliced = tf.slice(sequence, [pivot, 0], [end, -1])
	return sliced

def get_windows(sample, max_obs, binary=True):
	if binary:
		input_dict = deserialize(sample)
	else:
		input_dict = sample

	sequence = input_dict['input']
	rest = input_dict['length']%(max_obs)

	pivots = tf.tile([max_obs], [tf.cast(input_dict['length']/max_obs, tf.int32)]) # toma la cantidad de ventanas que se pueden formar dentro de la curvas de luz
																				   # puede haber trozos de curvas de luz menor al max_obs que no considerara
	pivots = tf.concat([[0], pivots], 0)
	pivots = tf.math.cumsum(pivots)

	splits = tf.map_fn(lambda x: get_window(sequence,
											input_dict['length'],
											x,
											max_obs),  pivots,
											infer_shape=False,
											fn_output_signature=(tf.float32))

	input_dict['label']  = tf.tile([input_dict['label']], [len(splits)])
	input_dict['lcid']   = tf.tile([input_dict['lcid']], [len(splits)])
	input_dict['length'] = tf.tile([input_dict['length']], [len(splits)])
	input_dict['input']  = splits

	return input_dict

all_dataset = dataset.map(lambda x: get_windows(x,
                                            max_obs=window_size,
                                            binary=False),
                        num_parallel_calls=tf.data.experimental.AUTOTUNE)

all_dataset = all_dataset.flat_map(lambda x: tf.data.Dataset.from_tensor_slices(x))

## Comparing get_sampling and get_windows

In [ ]:
dataset_sampling = get_data(samples_dataset)
dataset_sampling

b"b'23.3665.599'"
b"b'53.3115.350'"
b"b'9.5486.943'"
b"b'10.3560.4'"
b"b'17.2586.233'"
b"b'24.3585.23'"


,lcid,lc_data,label
0,"b""b'3.7081.943'""","[[49961.613, -4.848, 0.063], [49966.715, -5.01...",5
0,"b""b'10.3430.1344'""","[[49001.53, -4.481, 0.057], [49003.59, -4.546,...",5
0,"b""b'18.2598.715'""","[[49065.574, -4.85, 0.052], [49067.527, -5.029...",5
0,"b""b'10.3800.1073'""","[[49818.473, -4.98, 0.045], [49829.46, -4.818,...",5
0,"b""b'11.9109.1427'""","[[49135.367, -4.919, 0.054], [49144.37, -4.889...",5
...,...,...,...
0,"b""b'47.2134.31'""","[[49257.67, -7.959, 0.003], [49260.61, -7.858,...",2
0,"b""b'80.7079.1118'""","[[49311.72, -4.978, 0.085], [49312.516, -4.972...",2
0,"b""b'12.10922.199'""","[[49223.637, -5.937, 0.047], [49224.605, -5.93...",2
0,"b""b'11.8632.36'""","[[49699.695, -8.239, 0.021], [49700.527, -8.21...",2


In [ ]:
dataset_windows = get_data(all_dataset)
dataset_windows

b"b'23.3665.599'"
b"b'53.3115.350'"
b"b'9.5486.943'"
b"b'10.3560.4'"
b"b'17.2586.233'"
b"b'24.3585.23'"


,lcid,lc_data,label
0,"b""b'3.7081.943'""","[[48917.543, -5.061, 0.038], [48918.76, -5.039...",5
0,"b""b'3.7081.943'""","[[49461.51, -4.964, 0.059], [49462.477, -4.862...",5
0,"b""b'3.7081.943'""","[[50212.438, -5.266, 0.057], [50216.434, -4.90...",5
0,"b""b'3.7081.943'""","[[50335.71, -4.931, 0.041], [50349.68, -5.015,...",5
0,"b""b'10.3430.1344'""","[[48825.76, -4.808, 0.095], [48826.727, -4.876...",5
...,...,...,...
0,"b""b'6.7054.88'""","[[48823.67, -6.931, 0.038], [48824.68, -6.889,...",2
0,"b""b'6.7054.88'""","[[49162.766, -6.881, 0.025], [49163.38, -6.823...",2
0,"b""b'6.7054.88'""","[[49545.395, -6.862, 0.023], [49546.68, -6.87,...",2
0,"b""b'6.7054.88'""","[[50007.668, -6.871, 0.016], [50013.55, -6.939...",2


Seguiremos con sampling = True

In [ ]:
dataset_1 = all_dataset.map(lambda x: {'input' :x['input'],
                                       'original' :x['input'],
                                       'lcid'  :x['lcid'],
                                       'length':x['length'],
                                       'mask'  :tf.ones(tf.shape(x['input'])[0]),
                                       'label' : x['label']},
                                       num_parallel_calls=tf.data.experimental.AUTOTUNE)


sizes = {
		'input': (window_size, None),
		'original': (window_size, None),
		'lcid': (),
		'length': (),
		'mask': (window_size),
		'label': ()
	}


In [ ]:
def standardize(tensor, axis=0, return_mean=False):
    """
    Standardize a tensor subtracting the mean

    Args:
        tensor (1-dim tensorflow tensor): values
        axis (int): axis on which we calculate the mean
        return_mean (bool): output the mean of the tensor
                            turning on the original scale
    Returns:
        tensor (1-dim tensorflow tensor): standardize tensor
    """

    mean_value = tf.reduce_mean(tensor['input'], axis, name='mean_value')
    tensor['input'] = tensor['input'] - tf.expand_dims(mean_value, axis)

    mean_value = tf.reduce_mean(tensor['original'], axis, name='mean_value')
    tensor['original'] = tensor['original'] - tf.expand_dims(mean_value, axis)
    
    return tensor

In [ ]:
dataset_std = dataset_1.map(standardize)

In [ ]:
for lc_info in dataset_std:
    np_lc = lc_info['input'].numpy()
    np_lc = np_lc[np_lc[:,0].argsort()]

    if np_lc.shape[0] < 200:
        print(lc_info['lcid'].numpy())
        id_temporal = lc_info['lcid'].numpy()
        lc_temporal_info = copy.deepcopy(lc_info)

b"b'23.3665.599'"
b"b'53.3115.350'"
b"b'9.5486.943'"
b"b'10.3560.4'"
b"b'17.2586.233'"
b"b'24.3585.23'"


In [ ]:
lc_temporal_info['input'].numpy().shape

(178, 3)

In [ ]:
lc_temporal_info['mask']

<tf.Tensor: shape=(178,), dtype=float32, numpy=
array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)>

In [ ]:
dataset_padded = dataset_std.padded_batch(batch_size, padded_shapes=sizes)
dataset_padded

<_PaddedBatchDataset element_spec={'input': TensorSpec(shape=(None, 200, None), dtype=tf.float32, name=None), 'original': TensorSpec(shape=(None, 200, None), dtype=tf.float32, name=None), 'lcid': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'length': TensorSpec(shape=(None,), dtype=tf.int32, name=None), 'mask': TensorSpec(shape=(None, 200), dtype=tf.float32, name=None), 'label': TensorSpec(shape=(None,), dtype=tf.int32, name=None)}>

In [ ]:
for batch, lc_info_batches in enumerate(dataset_padded):
    print('Numero de batch {}'.format(batch))
    
    for snid, np_lc, mask_pad in zip(lc_info_batches['lcid'].numpy(), lc_info_batches['input'], lc_info_batches['mask']):
        #np_lc = np_lc[np_lc[:,0].argsort()]

        if snid == id_temporal:
            print('Encontramos la SNID')
            lc_temporal_padded = copy.deepcopy(np_lc)
            mask_padded = copy.deepcopy(mask_pad)
            lc_info_with_pad = copy.deepcopy(lc_info_batches)
    
        #print(lc_info['lcid'].numpy())
       # id_temporal = lc_info['lcid'].numpy()  
       # lc_temporal = copy.deepcopy(np_lc)

Numero de batch 0
Numero de batch 1
Numero de batch 2
Numero de batch 3
Numero de batch 4
Numero de batch 5
Numero de batch 6
Numero de batch 7
Numero de batch 8
Numero de batch 9
Numero de batch 10
Numero de batch 11
Numero de batch 12
Numero de batch 13
Numero de batch 14
Numero de batch 15
Numero de batch 16
Numero de batch 17
Numero de batch 18
Numero de batch 19
Numero de batch 20
Numero de batch 21
Numero de batch 22
Encontramos la SNID
Numero de batch 23
Numero de batch 24


In [ ]:
lc_info_batches.keys()

dict_keys(['input', 'original', 'lcid', 'length', 'mask', 'label'])

In [ ]:
print(lc_temporal_info['input'].shape)
print(lc_temporal_info['input'])

(178, 3)
tf.Tensor(
[[-1.44449609e+03 -8.21390152e-02  4.49437648e-05]
 [-1.43655078e+03 -8.71391296e-02  1.04494346e-03]
 [-1.42655469e+03 -1.24138832e-01 -9.55056399e-04]
 [-1.42550781e+03 -1.01139069e-01 -9.55056399e-04]
 [-1.42455469e+03 -1.04139328e-01  2.04494363e-03]
 [-1.34866016e+03 -6.81390762e-02 -1.95505633e-03]
 [-1.34072266e+03 -4.41389084e-02 -9.55056399e-04]
 [-1.33964453e+03 -2.01396942e-02 -1.95505633e-03]
 [-1.33874609e+03 -2.41394043e-02 -9.55056399e-04]
 [-1.33673047e+03 -7.13920593e-03 -1.95505633e-03]
 [-1.32967578e+03 -3.71389389e-02 -9.55056399e-04]
 [-1.25262891e+03  2.38609314e-02  4.49437648e-05]
 [-1.23079688e+03  1.01861000e-01 -1.95505633e-03]
 [-1.22463281e+03  9.18607712e-02 -9.55056399e-04]
 [-1.19973047e+03  1.25861168e-01  4.49437648e-05]
 [-1.18769141e+03  1.81860924e-01  1.04494346e-03]
 [-1.17279688e+03  2.70860672e-01  4.49437648e-05]
 [-1.16774609e+03  2.48860359e-01  2.60449424e-02]
 [-1.15981250e+03  3.43860626e-01  7.04494352e-03]
 [-1.155812

In [ ]:
print(lc_temporal_padded.shape)
print(lc_temporal_padded)

(200, 3)
tf.Tensor(
[[-1.44449609e+03 -8.21390152e-02  4.49437648e-05]
 [-1.43655078e+03 -8.71391296e-02  1.04494346e-03]
 [-1.42655469e+03 -1.24138832e-01 -9.55056399e-04]
 [-1.42550781e+03 -1.01139069e-01 -9.55056399e-04]
 [-1.42455469e+03 -1.04139328e-01  2.04494363e-03]
 [-1.34866016e+03 -6.81390762e-02 -1.95505633e-03]
 [-1.34072266e+03 -4.41389084e-02 -9.55056399e-04]
 [-1.33964453e+03 -2.01396942e-02 -1.95505633e-03]
 [-1.33874609e+03 -2.41394043e-02 -9.55056399e-04]
 [-1.33673047e+03 -7.13920593e-03 -1.95505633e-03]
 [-1.32967578e+03 -3.71389389e-02 -9.55056399e-04]
 [-1.25262891e+03  2.38609314e-02  4.49437648e-05]
 [-1.23079688e+03  1.01861000e-01 -1.95505633e-03]
 [-1.22463281e+03  9.18607712e-02 -9.55056399e-04]
 [-1.19973047e+03  1.25861168e-01  4.49437648e-05]
 [-1.18769141e+03  1.81860924e-01  1.04494346e-03]
 [-1.17279688e+03  2.70860672e-01  4.49437648e-05]
 [-1.16774609e+03  2.48860359e-01  2.60449424e-02]
 [-1.15981250e+03  3.43860626e-01  7.04494352e-03]
 [-1.155812

In [ ]:
mask_padded

<tf.Tensor: shape=(200,), dtype=float32, numpy=
array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>

In [ ]:
# MASKING

def get_probed(input_dict, probed, njobs):

    input_shape = tf.shape(input_dict['input'])

    if probed == 1.:
        probed_mask = tf.ones([input_shape[0], input_shape[1]]) * input_dict['mask']
        input_dict['probed_mask']  = probed_mask
        input_dict['att_mask'] = 1. - probed_mask
        return input_dict

    nprobed = tf.multiply(tf.cast(input_shape[1], tf.float32), probed)
    nprobed = tf.cast(nprobed, tf.int32)
    random_integers = tf.range(input_shape[1], dtype=tf.int32)
    indices = tf.map_fn(lambda x: tf.random.shuffle(random_integers), 
                                      tf.range(input_shape[0]),
                                      parallel_iterations=njobs)
    indices = tf.slice(indices, [0, 0], [-1, nprobed])
    random_mask = tf.one_hot(indices, input_shape[1])
    random_mask = tf.reduce_sum(random_mask, 1)

    input_dict['probed_mask'] = random_mask * input_dict['mask']
    att_mask = (1 - input_dict['mask']) + random_mask 
    att_mask = tf.minimum(att_mask, 1)
    input_dict['att_mask'] = att_mask

    return input_dict

dataset_probed = dataset_padded.map(lambda x: get_probed(x, probed=probed, njobs=njobs))

In [ ]:
input_shape = tf.shape(lc_info_batches['input'])
input_shape[1]

<tf.Tensor: shape=(), dtype=int32, numpy=200>

In [ ]:
nprobed = tf.multiply(tf.cast(input_shape[1], tf.float32), probed)
nprobed = tf.cast(nprobed, tf.int32)
nprobed

<tf.Tensor: shape=(), dtype=int32, numpy=80>

In [ ]:
random_integers = tf.range(input_shape[1], dtype=tf.int32)
random_integers

<tf.Tensor: shape=(200,), dtype=int32, numpy=
array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 1

In [ ]:
indices = tf.map_fn(lambda x: tf.random.shuffle(random_integers),
                                        tf.range(input_shape[0]),
                                        parallel_iterations=njobs)

indices

<tf.Tensor: shape=(14, 200), dtype=int32, numpy=
array([[ 75, 130,  81, ..., 194, 144, 125],
       [ 29,  93,  40, ...,  46, 146,   8],
       [152,  10,  72, ...,  28,  23,  97],
       ...,
       [ 31,   2,  18, ..., 152,  69, 174],
       [116,  21,  69, ...,  81,  59,  29],
       [198,  54,  68, ..., 165,   5, 164]], dtype=int32)>

In [ ]:
np.unique(indices[0]).shape

(200,)

In [ ]:
indices = tf.slice(indices, [0, 0], [-1, nprobed])
indices

<tf.Tensor: shape=(14, 80), dtype=int32, numpy=
array([[ 75, 130,  81, ...,  47,  90,  23],
       [ 29,  93,  40, ..., 133,  57, 107],
       [152,  10,  72, ...,  95,  30, 102],
       ...,
       [ 31,   2,  18, ..., 155,  40,  84],
       [116,  21,  69, ...,  12,  57,  93],
       [198,  54,  68, ..., 107, 139, 192]], dtype=int32)>

In [ ]:
random_mask = tf.one_hot(indices, input_shape[1])
random_mask

<tf.Tensor: shape=(14, 80, 200), dtype=float32, numpy=
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0.,

In [ ]:
# 1 los valores probed (80)
# 0 los valores no probed (120)

random_mask = tf.reduce_sum(random_mask, 1)
random_mask

<tf.Tensor: shape=(14, 200), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 1., 1.],
       [1., 1., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 1., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 1., 0.]], dtype=float32)>

In [ ]:
lc_info_batches.keys()

dict_keys(['input', 'original', 'lcid', 'length', 'mask', 'label'])

In [ ]:
lc_info_batches['mask']

<tf.Tensor: shape=(14, 200), dtype=float32, numpy=
array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)>

In [ ]:
lc_info_batches['probed_mask'] = random_mask * lc_info_batches['mask']
lc_info_batches['probed_mask'] 

<tf.Tensor: shape=(14, 200), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 1., 1.],
       [1., 1., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 1., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 1., 0.]], dtype=float32)>

In [ ]:
## Comprobacion
for lc_probed_mask in lc_info_batches['probed_mask']:
    print(np.sum(lc_probed_mask))

80.0
80.0
80.0
80.0
80.0
80.0
80.0
80.0
80.0
80.0
80.0
80.0
80.0
80.0


In [ ]:
lc_info_with_pad['mask']

<tf.Tensor: shape=(16, 200), dtype=float32, numpy=
array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)>

In [ ]:
## Comprobacion
def check_probed(lc_batch):

    input_shape = tf.shape(lc_batch['input'])
    print(input_shape[1])

    nprobed = tf.multiply(tf.cast(input_shape[1], tf.float32), probed)
    nprobed = tf.cast(nprobed, tf.int32)
    print(nprobed)

    random_integers = tf.range(input_shape[1], dtype=tf.int32)
    print(random_integers)

    indices = tf.map_fn(lambda x: tf.random.shuffle(random_integers),
                                            tf.range(input_shape[0]),
                                            parallel_iterations=njobs)

    print(indices)

    indices = tf.slice(indices, [0, 0], [-1, nprobed])
    print(indices)

    random_mask = tf.one_hot(indices, input_shape[1])
    random_mask = tf.reduce_sum(random_mask, 1)
    print(random_mask)
    print('*'*20)

    lc_batch['probed_mask'] = random_mask * lc_batch['mask']
    for lc_probed_mask in lc_info_with_pad['probed_mask']:
        print(np.sum(lc_probed_mask))
        

check_probed(lc_info_with_pad)

tf.Tensor(200, shape=(), dtype=int32)
tf.Tensor(80, shape=(), dtype=int32)
tf.Tensor(
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199], shape=(200,), dtype=int32)
tf.Tensor(
[[ 23 198  98 ... 159 175 186]
 [ 74  52  24 ...  26 147   4]


In [ ]:
## Comprobacion
def check_probed_mask_pad(lc_batch):

    input_shape = tf.shape(lc_batch['input'])
    print(input_shape[1])

    nprobed = tf.multiply(tf.cast(input_shape[1], tf.float32), probed)
    nprobed = tf.cast(nprobed, tf.int32)
    print(nprobed)

    random_integers = tf.range(input_shape[1], dtype=tf.int32)
    print(random_integers)

    indices = tf.map_fn(lambda x: tf.random.shuffle(random_integers),
                                            tf.range(input_shape[0]),
                                            parallel_iterations=njobs)

    print(indices)

    indices = tf.slice(indices, [0, 0], [-1, nprobed])
    print(indices)

    random_mask = tf.one_hot(indices, input_shape[1])
    random_mask = tf.reduce_sum(random_mask, 1)
    print(random_mask)
    print('*'*20)

    lc_batch['probed_mask'] = random_mask * lc_batch['mask']
    for lc_probed_mask in lc_info_with_pad['probed_mask']:
        print(np.sum(lc_probed_mask))

    att_mask = (1 - lc_batch['mask']) + random_mask 
    
    return att_mask
        
att_mask_padded = check_probed_mask_pad(lc_info_with_pad)
print(att_mask_padded)
att_mask_padded = tf.minimum(att_mask_padded, 1)
print(att_mask_padded)

tf.Tensor(200, shape=(), dtype=int32)
tf.Tensor(80, shape=(), dtype=int32)
tf.Tensor(
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199], shape=(200,), dtype=int32)
tf.Tensor(
[[129  93 135 ... 177  88 194]
 [184 116 144 ...  44 176 149]


In [ ]:
att_mask = (1 - lc_info_batches['mask']) + random_mask 
att_mask

<tf.Tensor: shape=(14, 200), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 1., 1.],
       [1., 1., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 1., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 1., 0.]], dtype=float32)>

In [ ]:
lc_info_batches['probed_mask'] 

<tf.Tensor: shape=(14, 200), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 1., 1.],
       [1., 1., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 1., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 1., 0.]], dtype=float32)>

In [ ]:
###################################################### Corrobora Innecesarioamente lo mismo
att_mask == lc_info_batches['probed_mask']

<tf.Tensor: shape=(14, 200), dtype=bool, numpy=
array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])>

In [ ]:
np.sum(lc_info_batches['probed_mask'][0])

80.0

In [ ]:
# Corrobora Innecesarioamente lo mismo
att_mask == tf.minimum(att_mask, 1)

<tf.Tensor: shape=(14, 200), dtype=bool, numpy=
array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])>

In [ ]:
# Innecesarioamente lo mismo
att_mask = tf.minimum(att_mask, 1)
att_mask

<tf.Tensor: shape=(14, 200), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 1., 1.],
       [1., 1., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 1., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 1., 0.]], dtype=float32)>

In [ ]:
lc_info_batches['att_mask'] = att_mask
lc_info_batches['att_mask']

<tf.Tensor: shape=(14, 200), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 1., 1.],
       [1., 1., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 1., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 1., 0.]], dtype=float32)>

In [ ]:
def create_mask(pre_mask, n_elements):
    indices = tf.where(pre_mask)
    indices = tf.random.shuffle(indices)
    indices = tf.slice(indices, [0, 0], [n_elements, -1])

    mask = tf.one_hot(indices, tf.shape(pre_mask)[0], dtype=tf.int32)
    mask = tf.reduce_sum(mask, 0)
    mask = tf.reshape(mask, [tf.shape(pre_mask)[0]])
    
    return mask


def add_random(input_dict, random_frac, njobs):
    """ Add random observations to each sequence
        
    Args:
        random_frac (number): Fraction of probed (in decimal) to be replaced with random values
    """ 
    input_shape = tf.shape(input_dict['input'])
    input_dict['input_pre_nsp'] = input_dict['input'] 

    # ====== RANDOM MASK =====
    n_probed = tf.reduce_sum(input_dict['probed_mask'], 1)
    n_random = tf.math.ceil(n_probed * random_frac)
    n_random = tf.cast(n_random, tf.int32)
    random_mask = tf.map_fn(lambda x: create_mask(x[0], x[1]),
                                (input_dict['probed_mask'], n_random),
                                parallel_iterations=njobs,
                                fn_output_signature=tf.int32)

    # ====== SAME MASK =====
    rest = tf.cast(input_dict['probed_mask'], tf.int32) * (1-random_mask)
    n_rest = tf.reduce_sum(rest, 1)
    n_same = tf.math.ceil(tf.cast(n_rest, tf.float32) * random_frac)
    n_same = tf.cast(n_same, tf.int32)

    same_mask = tf.map_fn(lambda x: create_mask(x[0], x[1]), 
                                  (rest, n_same),
                                  parallel_iterations=njobs,
                                  fn_output_signature=tf.int32)

    # ===== REPLACEMENT ==== 
    random_replacement = tf.random.shuffle(tf.transpose(input_dict['input'], [1, 0, 2]))
    random_replacement = tf.transpose(random_replacement, [1, 0, 2])
    random_replacement = random_replacement * tf.cast(tf.expand_dims(random_mask, -1), tf.float32) * [0., 1., 1.]

    # Mask refering to observations that do not change
    keep_mask = tf.expand_dims(1 - random_mask, -1)
    keep_mask = tf.tile(keep_mask, [1, 1, input_shape[-1]-1])
    keep_mask = tf.concat([tf.zeros([input_shape[0], input_shape[1], 1], dtype=tf.int32), keep_mask], 2)
    keep_mask = tf.abs([1, 0, 0] - keep_mask)

    # Part of the input we mantain
    keep_input = input_dict['input'] * tf.cast(keep_mask, tf.float32)

    # Replacing original input with the randomized one
    input_dict['input']  = random_replacement + keep_input

    # Attention mask is 1 when masked. 
    # Random mask is 1 for masked observations selected to be randomized
    # then,
    att_mask    = tf.cast(input_dict['att_mask'], tf.bool)
    random_mask = tf.cast(random_mask, tf.bool)
    same_mask   = tf.cast(same_mask, tf.bool)

    att_mask = tf.math.logical_xor(att_mask, random_mask)
    att_mask = tf.math.logical_xor(att_mask, same_mask)

    input_dict['att_mask'] = tf.cast(att_mask, tf.float32)

    return input_dict

dataset_random_mask = dataset_probed.map(lambda x: add_random(x, random_frac=random_same, njobs=njobs))

In [ ]:
random_frac=random_same
random_frac

0.2

In [ ]:
input_shape = tf.shape(lc_info_batches['input'])
lc_info_batches['input_pre_nsp'] = lc_info_batches['input']

In [ ]:
lc_info_batches['probed_mask']

<tf.Tensor: shape=(14, 200), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 1., 1.],
       [1., 1., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 1., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 1., 0.]], dtype=float32)>

In [ ]:
# ====== RANDOM MASK =====
n_probed = tf.reduce_sum(lc_info_batches['probed_mask'], 1)
n_probed

<tf.Tensor: shape=(14,), dtype=float32, numpy=
array([80., 80., 80., 80., 80., 80., 80., 80., 80., 80., 80., 80., 80.,
       80.], dtype=float32)>

In [ ]:
n_random = tf.math.ceil(n_probed * random_frac)
n_random = tf.cast(n_random, tf.int32)
n_random

<tf.Tensor: shape=(14,), dtype=int32, numpy=
array([16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16],
      dtype=int32)>

In [ ]:
def create_mask(pre_mask, n_elements):
    print(pre_mask.shape)
    print(n_elements)
    indices = tf.where(pre_mask)
    indices = tf.random.shuffle(indices)
    indices = tf.slice(indices, [0, 0], [n_elements, -1])

    mask = tf.one_hot(indices, tf.shape(pre_mask)[0], dtype=tf.int32)
    mask = tf.reduce_sum(mask, 0)
    mask = tf.reshape(mask, [tf.shape(pre_mask)[0]])
    
    return mask

random_mask = tf.map_fn(lambda x: create_mask(x[0], x[1]),
                            (lc_info_batches['probed_mask'], n_random),
                            parallel_iterations=njobs,
                            fn_output_signature=tf.int32)

random_mask

(200,)
tf.Tensor(16, shape=(), dtype=int32)
(200,)
tf.Tensor(16, shape=(), dtype=int32)
(200,)
tf.Tensor(16, shape=(), dtype=int32)
(200,)
tf.Tensor(16, shape=(), dtype=int32)
(200,)
tf.Tensor(16, shape=(), dtype=int32)
(200,)
tf.Tensor(16, shape=(), dtype=int32)
(200,)
tf.Tensor(16, shape=(), dtype=int32)
(200,)
tf.Tensor(16, shape=(), dtype=int32)
(200,)
tf.Tensor(16, shape=(), dtype=int32)
(200,)
tf.Tensor(16, shape=(), dtype=int32)
(200,)
tf.Tensor(16, shape=(), dtype=int32)
(200,)
tf.Tensor(16, shape=(), dtype=int32)
(200,)
tf.Tensor(16, shape=(), dtype=int32)
(200,)
tf.Tensor(16, shape=(), dtype=int32)


<tf.Tensor: shape=(14, 200), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 1, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 1, 0]], dtype=int32)>

In [ ]:
pre_mask = lc_info_batches['probed_mask'][0]
n_elements = n_random[0]
pre_mask

<tf.Tensor: shape=(200,), dtype=float32, numpy=
array([0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0.,
       0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1.,
       0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1.,
       0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0.,
       1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 1.,
       0., 1., 1., 0., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0.,
       0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.,
       0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1.,
       0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1.], dtype=float32)>

In [ ]:
n_elements

<tf.Tensor: shape=(), dtype=int32, numpy=16>

In [ ]:
indices = tf.where(pre_mask)
indices

<tf.Tensor: shape=(80, 1), dtype=int64, numpy=
array([[  4],
       [  5],
       [  6],
       [  7],
       [  8],
       [  9],
       [ 10],
       [ 14],
       [ 18],
       [ 23],
       [ 24],
       [ 26],
       [ 27],
       [ 29],
       [ 31],
       [ 33],
       [ 40],
       [ 41],
       [ 43],
       [ 47],
       [ 48],
       [ 49],
       [ 50],
       [ 54],
       [ 56],
       [ 59],
       [ 60],
       [ 62],
       [ 63],
       [ 66],
       [ 68],
       [ 74],
       [ 75],
       [ 76],
       [ 77],
       [ 81],
       [ 82],
       [ 84],
       [ 86],
       [ 87],
       [ 89],
       [ 90],
       [ 93],
       [ 94],
       [ 96],
       [ 98],
       [103],
       [104],
       [111],
       [112],
       [113],
       [114],
       [115],
       [116],
       [130],
       [131],
       [133],
       [135],
       [137],
       [139],
       [141],
       [150],
       [152],
       [157],
       [158],
       [160],
       [164],
       [167],
 

In [ ]:
indices = tf.random.shuffle(indices)
indices

<tf.Tensor: shape=(80, 1), dtype=int64, numpy=
array([[ 82],
       [ 48],
       [ 23],
       [ 54],
       [ 96],
       [158],
       [ 66],
       [131],
       [193],
       [ 75],
       [ 43],
       [ 81],
       [150],
       [182],
       [ 59],
       [ 74],
       [186],
       [103],
       [  7],
       [ 93],
       [ 68],
       [174],
       [ 62],
       [ 84],
       [  9],
       [  6],
       [157],
       [198],
       [170],
       [167],
       [135],
       [ 27],
       [ 40],
       [ 89],
       [ 49],
       [160],
       [115],
       [195],
       [141],
       [152],
       [ 29],
       [199],
       [ 56],
       [ 87],
       [179],
       [188],
       [111],
       [114],
       [  8],
       [133],
       [190],
       [ 98],
       [ 33],
       [  5],
       [130],
       [ 18],
       [ 41],
       [ 14],
       [ 26],
       [113],
       [ 77],
       [164],
       [ 76],
       [183],
       [116],
       [112],
       [ 47],
       [104],
 

In [ ]:
indices = tf.slice(indices, [0, 0], [n_elements, -1])
indices

<tf.Tensor: shape=(16, 1), dtype=int64, numpy=
array([[ 82],
       [ 48],
       [ 23],
       [ 54],
       [ 96],
       [158],
       [ 66],
       [131],
       [193],
       [ 75],
       [ 43],
       [ 81],
       [150],
       [182],
       [ 59],
       [ 74]])>

In [ ]:
mask = tf.one_hot(indices, tf.shape(pre_mask)[0], dtype=tf.int32)
mask

<tf.Tensor: shape=(16, 1, 200), dtype=int32, numpy=
array([[[0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0]]], dtype=int32)>

In [ ]:
mask = tf.reduce_sum(mask, 0)
mask

<tf.Tensor: shape=(1, 200), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0]], dtype=int32)>

In [ ]:
mask = tf.reshape(mask, [tf.shape(pre_mask)[0]])
mask

<tf.Tensor: shape=(200,), dtype=int32, numpy=
array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0], dtype=int32)>

In [ ]:
random_mask

<tf.Tensor: shape=(14, 200), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 1, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 1, 0]], dtype=int32)>

In [ ]:
# ====== SAME MASK =====
rest = tf.cast(lc_info_batches['probed_mask'], tf.int32) * (1-random_mask)
rest

<tf.Tensor: shape=(14, 200), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 1, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=int32)>

In [ ]:
n_rest = tf.reduce_sum(rest, 1)
n_rest

<tf.Tensor: shape=(14,), dtype=int32, numpy=
array([64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64],
      dtype=int32)>

In [ ]:
n_same = tf.math.ceil(tf.cast(n_rest, tf.float32) * random_frac)
n_same

<tf.Tensor: shape=(14,), dtype=float32, numpy=
array([13., 13., 13., 13., 13., 13., 13., 13., 13., 13., 13., 13., 13.,
       13.], dtype=float32)>

In [ ]:
n_same = tf.cast(n_same, tf.int32)
n_same

<tf.Tensor: shape=(14,), dtype=int32, numpy=
array([13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13],
      dtype=int32)>

In [ ]:
same_mask = tf.map_fn(lambda x: create_mask(x[0], x[1]), 
                                (rest, n_same),
                                parallel_iterations=njobs,
                                fn_output_signature=tf.int32)

same_mask

(200,)
tf.Tensor(13, shape=(), dtype=int32)
(200,)
tf.Tensor(13, shape=(), dtype=int32)
(200,)
tf.Tensor(13, shape=(), dtype=int32)
(200,)
tf.Tensor(13, shape=(), dtype=int32)
(200,)
tf.Tensor(13, shape=(), dtype=int32)
(200,)
tf.Tensor(13, shape=(), dtype=int32)
(200,)
tf.Tensor(13, shape=(), dtype=int32)
(200,)
tf.Tensor(13, shape=(), dtype=int32)
(200,)
tf.Tensor(13, shape=(), dtype=int32)
(200,)
tf.Tensor(13, shape=(), dtype=int32)
(200,)
tf.Tensor(13, shape=(), dtype=int32)
(200,)
tf.Tensor(13, shape=(), dtype=int32)
(200,)
tf.Tensor(13, shape=(), dtype=int32)
(200,)
tf.Tensor(13, shape=(), dtype=int32)


<tf.Tensor: shape=(14, 200), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>

In [ ]:
lc_info_batches['input']

<tf.Tensor: shape=(14, 200, 3), dtype=float32, numpy=
array([[[-2.6617578e+02, -5.5631161e-02, -2.0884996e-02],
        [-2.6527734e+02,  1.5836906e-01, -1.5884995e-02],
        [-2.6414844e+02,  3.2368660e-02,  5.1150061e-03],
        ...,
        [ 2.7077344e+02, -3.7631035e-02, -1.8884996e-02],
        [ 2.7179297e+02, -2.0631313e-02, -1.6884996e-02],
        [ 2.7966406e+02,  1.8636894e-01, -1.9884996e-02]],

       [[-3.5253906e+02, -1.5975475e-02, -4.3549910e-03],
        [-3.4756641e+02, -1.4797544e-01,  3.5645012e-02],
        [-3.4554297e+02,  1.2502480e-01,  2.6450083e-03],
        ...,
        [ 2.9165625e+02,  5.8024883e-02, -1.5354991e-02],
        [ 2.9549219e+02, -2.5975227e-02, -1.6354991e-02],
        [ 2.9651172e+02, -5.5975437e-02, -3.3549927e-03]],

       [[-4.9218359e+02, -7.2044373e-02,  1.6059995e-02],
        [-4.8922266e+02, -8.2044125e-02, -1.9400045e-03],
        [-4.8826953e+02,  2.9955864e-02, -9.9400058e-03],
        ...,
        [ 5.8973438e+02,  3.39560

In [ ]:
tf.transpose(lc_info_batches['input'], [1, 0, 2])

<tf.Tensor: shape=(200, 14, 3), dtype=float32, numpy=
array([[[-2.6617578e+02, -5.5631161e-02, -2.0884996e-02],
        [-3.5253906e+02, -1.5975475e-02, -4.3549910e-03],
        [-4.9218359e+02, -7.2044373e-02,  1.6059995e-02],
        ...,
        [-1.9699219e+02,  5.3181648e-03,  4.2449962e-03],
        [-2.7599609e+02,  2.9764175e-03, -5.4999255e-04],
        [-5.0294141e+02,  1.6838074e-02, -8.4150052e-03]],

       [[-2.6527734e+02,  1.5836906e-01, -1.5884995e-02],
        [-3.4756641e+02, -1.4797544e-01,  3.5645012e-02],
        [-4.8922266e+02, -8.2044125e-02, -1.9400045e-03],
        ...,
        [-1.9570703e+02, -2.6817322e-03,  1.2449957e-03],
        [-2.7011328e+02, -6.5023899e-02, -2.5499929e-03],
        [-5.0098828e+02,  2.5838375e-02, -4.4150054e-03]],

       [[-2.6414844e+02,  3.2368660e-02,  5.1150061e-03],
        [-3.4554297e+02,  1.2502480e-01,  2.6450083e-03],
        [-4.8826953e+02,  2.9955864e-02, -9.9400058e-03],
        ...,
        [-1.9371094e+02,  5.63182

In [ ]:
# ===== REPLACEMENT ==== 
random_replacement = tf.random.shuffle(tf.transpose(lc_info_batches['input'], [1, 0, 2]))
random_replacement

<tf.Tensor: shape=(200, 14, 3), dtype=float32, numpy=
array([[[ 2.16882812e+02, -1.26309395e-02, -1.48849953e-02],
        [ 2.53449219e+02, -5.69753647e-02, -2.33549904e-02],
        [ 4.98863281e+02, -6.30440712e-02, -1.39400065e-02],
        ...,
        [ 2.11246094e+02,  1.33180618e-02, -5.75500354e-03],
        [ 1.77863281e+02,  6.97612762e-03, -9.54999309e-03],
        [ 4.76171875e+02,  1.48838043e-01, -5.41500561e-03]],

       [[ 9.68437500e+01,  4.36878204e-03,  3.81150059e-02],
        [ 1.81531250e+02,  1.18024826e-01,  2.06450075e-02],
        [ 3.24640625e+02, -2.50439644e-02, -1.49400067e-02],
        ...,
        [ 1.49000000e+02, -6.81877136e-04, -7.75500387e-03],
        [ 1.26933594e+02, -3.02362442e-03, -5.49992546e-04],
        [ 2.93144531e+02,  1.98383331e-02, -7.41500594e-03]],

       [[-1.13171875e+02,  1.19369030e-01, -1.98849961e-02],
        [-1.91464844e+02, -9.49754715e-02,  1.36450082e-02],
        [-3.23375000e+02,  3.99560928e-02, -4.94000688e-03],
 

In [ ]:
random_replacement = tf.transpose(random_replacement, [1, 0, 2])
random_replacement

<tf.Tensor: shape=(14, 200, 3), dtype=float32, numpy=
array([[[ 2.16882812e+02, -1.26309395e-02, -1.48849953e-02],
        [ 9.68437500e+01,  4.36878204e-03,  3.81150059e-02],
        [-1.13171875e+02,  1.19369030e-01, -1.98849961e-02],
        ...,
        [ 8.99101562e+01, -5.96313477e-02, -7.88499415e-03],
        [-7.22929688e+01, -2.66313553e-02, -1.88849960e-02],
        [ 1.34941406e+02, -4.16312218e-02,  1.15003437e-04]],

       [[ 2.53449219e+02, -5.69753647e-02, -2.33549904e-02],
        [ 1.81531250e+02,  1.18024826e-01,  2.06450075e-02],
        [-1.91464844e+02, -9.49754715e-02,  1.36450082e-02],
        ...,
        [ 1.70691406e+02,  2.30245590e-02,  1.46450102e-02],
        [-7.04296875e+01, -4.09750938e-02, -1.13549903e-02],
        [ 2.17453125e+02, -9.97543335e-03, -2.33549904e-02]],

       [[ 4.98863281e+02, -6.30440712e-02, -1.39400065e-02],
        [ 3.24640625e+02, -2.50439644e-02, -1.49400067e-02],
        [-3.23375000e+02,  3.99560928e-02, -4.94000688e-03],
 

In [ ]:
tf.cast(tf.expand_dims(random_mask, -1), tf.float32)

<tf.Tensor: shape=(14, 200, 1), dtype=float32, numpy=
array([[[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [1.],
        [0.]],

       [[1.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [1.],
        [0.],
        [0.]],

       ...,

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [1.],
        ...,
        [0.],
        [1.],
        [0.]]], dtype=float32)>

In [ ]:
tf.cast(tf.expand_dims(random_mask, -1), tf.float32) * [0., 1., 1.]

<tf.Tensor: shape=(14, 200, 3), dtype=float32, numpy=
array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 1., 1.],
        [0., 0., 0.]],

       [[0., 1., 1.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 1., 1.],
        [0., 0., 0.],
        [0., 0., 0.]],

       ...,

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 1., 1.],
        ...,
        [0., 0., 0.],
        [0., 1., 1.],
        [0., 0., 0.]]], dtype=float32)>

In [ ]:
random_replacement = random_replacement * tf.cast(tf.expand_dims(random_mask, -1), tf.float32) * [0., 1., 1.]
random_replacement

<tf.Tensor: shape=(14, 200, 3), dtype=float32, numpy=
array([[[ 0.        , -0.        , -0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [-0.        ,  0.        , -0.        ],
        ...,
        [ 0.        , -0.        , -0.        ],
        [-0.        , -0.02663136, -0.018885  ],
        [ 0.        , -0.        ,  0.        ]],

       [[ 0.        , -0.05697536, -0.02335499],
        [ 0.        ,  0.        ,  0.        ],
        [-0.        , -0.        ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        ],
        [-0.        , -0.        , -0.        ],
        [ 0.        , -0.        , -0.        ]],

       [[ 0.        , -0.        , -0.        ],
        [ 0.        , -0.        , -0.        ],
        [-0.        ,  0.        , -0.        ],
        ...,
        [ 0.        , -0.10904408, -0.01794001],
        [-0.        ,  0.        , -0.        ],
        [ 0.        , -0.        , -0.        ]],

       ...,

      

In [ ]:
np.sum((1 - random_mask)[0])

184

In [ ]:
# Mask refering to observations that do not change
keep_mask = tf.expand_dims(1 - random_mask, -1)
keep_mask

<tf.Tensor: shape=(14, 200, 1), dtype=int32, numpy=
array([[[1],
        [1],
        [1],
        ...,
        [1],
        [0],
        [1]],

       [[0],
        [1],
        [1],
        ...,
        [1],
        [1],
        [1]],

       [[1],
        [1],
        [1],
        ...,
        [0],
        [1],
        [1]],

       ...,

       [[1],
        [1],
        [1],
        ...,
        [1],
        [1],
        [1]],

       [[1],
        [1],
        [1],
        ...,
        [1],
        [1],
        [1]],

       [[1],
        [1],
        [0],
        ...,
        [1],
        [0],
        [1]]], dtype=int32)>

In [ ]:
input_shape[-1] - 1

<tf.Tensor: shape=(), dtype=int32, numpy=2>

In [ ]:
keep_mask = tf.tile(keep_mask, [1, 1, input_shape[-1]-1])
keep_mask

<tf.Tensor: shape=(14, 200, 2), dtype=int32, numpy=
array([[[1, 1],
        [1, 1],
        [1, 1],
        ...,
        [1, 1],
        [0, 0],
        [1, 1]],

       [[0, 0],
        [1, 1],
        [1, 1],
        ...,
        [1, 1],
        [1, 1],
        [1, 1]],

       [[1, 1],
        [1, 1],
        [1, 1],
        ...,
        [0, 0],
        [1, 1],
        [1, 1]],

       ...,

       [[1, 1],
        [1, 1],
        [1, 1],
        ...,
        [1, 1],
        [1, 1],
        [1, 1]],

       [[1, 1],
        [1, 1],
        [1, 1],
        ...,
        [1, 1],
        [1, 1],
        [1, 1]],

       [[1, 1],
        [1, 1],
        [0, 0],
        ...,
        [1, 1],
        [0, 0],
        [1, 1]]], dtype=int32)>

In [ ]:
tf.zeros([input_shape[0], input_shape[1], 1], dtype=tf.int32)

<tf.Tensor: shape=(14, 200, 1), dtype=int32, numpy=
array([[[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       [[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       [[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       ...,

       [[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       [[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       [[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]]], dtype=int32)>

In [ ]:
keep_mask = tf.concat([tf.zeros([input_shape[0], input_shape[1], 1], dtype=tf.int32), keep_mask], 2)
keep_mask

<tf.Tensor: shape=(14, 200, 3), dtype=int32, numpy=
array([[[0, 1, 1],
        [0, 1, 1],
        [0, 1, 1],
        ...,
        [0, 1, 1],
        [0, 0, 0],
        [0, 1, 1]],

       [[0, 0, 0],
        [0, 1, 1],
        [0, 1, 1],
        ...,
        [0, 1, 1],
        [0, 1, 1],
        [0, 1, 1]],

       [[0, 1, 1],
        [0, 1, 1],
        [0, 1, 1],
        ...,
        [0, 0, 0],
        [0, 1, 1],
        [0, 1, 1]],

       ...,

       [[0, 1, 1],
        [0, 1, 1],
        [0, 1, 1],
        ...,
        [0, 1, 1],
        [0, 1, 1],
        [0, 1, 1]],

       [[0, 1, 1],
        [0, 1, 1],
        [0, 1, 1],
        ...,
        [0, 1, 1],
        [0, 1, 1],
        [0, 1, 1]],

       [[0, 1, 1],
        [0, 1, 1],
        [0, 0, 0],
        ...,
        [0, 1, 1],
        [0, 0, 0],
        [0, 1, 1]]], dtype=int32)>

In [ ]:
keep_mask = tf.abs([1, 0, 0] - keep_mask)
keep_mask

<tf.Tensor: shape=(14, 200, 3), dtype=int32, numpy=
array([[[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        ...,
        [1, 1, 1],
        [1, 0, 0],
        [1, 1, 1]],

       [[1, 0, 0],
        [1, 1, 1],
        [1, 1, 1],
        ...,
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]],

       [[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        ...,
        [1, 0, 0],
        [1, 1, 1],
        [1, 1, 1]],

       ...,

       [[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        ...,
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]],

       [[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        ...,
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]],

       [[1, 1, 1],
        [1, 1, 1],
        [1, 0, 0],
        ...,
        [1, 1, 1],
        [1, 0, 0],
        [1, 1, 1]]], dtype=int32)>

In [ ]:
np.sum(keep_mask[0][:,1])

184

In [ ]:
# Part of the input we mantain
keep_input = lc_info_batches['input'] * tf.cast(keep_mask, tf.float32)
keep_input

<tf.Tensor: shape=(14, 200, 3), dtype=float32, numpy=
array([[[-2.6617578e+02, -5.5631161e-02, -2.0884996e-02],
        [-2.6527734e+02,  1.5836906e-01, -1.5884995e-02],
        [-2.6414844e+02,  3.2368660e-02,  5.1150061e-03],
        ...,
        [ 2.7077344e+02, -3.7631035e-02, -1.8884996e-02],
        [ 2.7179297e+02, -0.0000000e+00, -0.0000000e+00],
        [ 2.7966406e+02,  1.8636894e-01, -1.9884996e-02]],

       [[-3.5253906e+02, -0.0000000e+00, -0.0000000e+00],
        [-3.4756641e+02, -1.4797544e-01,  3.5645012e-02],
        [-3.4554297e+02,  1.2502480e-01,  2.6450083e-03],
        ...,
        [ 2.9165625e+02,  5.8024883e-02, -1.5354991e-02],
        [ 2.9549219e+02, -2.5975227e-02, -1.6354991e-02],
        [ 2.9651172e+02, -5.5975437e-02, -3.3549927e-03]],

       [[-4.9218359e+02, -7.2044373e-02,  1.6059995e-02],
        [-4.8922266e+02, -8.2044125e-02, -1.9400045e-03],
        [-4.8826953e+02,  2.9955864e-02, -9.9400058e-03],
        ...,
        [ 5.8973438e+02,  0.00000

In [ ]:
# Replacing original input with the randomized one
lc_info_batches['input']  = random_replacement + keep_input
lc_info_batches['input'] 

<tf.Tensor: shape=(14, 200, 3), dtype=float32, numpy=
array([[[-2.66175781e+02, -5.56311607e-02, -2.08849963e-02],
        [-2.65277344e+02,  1.58369064e-01, -1.58849955e-02],
        [-2.64148438e+02,  3.23686600e-02,  5.11500612e-03],
        ...,
        [ 2.70773438e+02, -3.76310349e-02, -1.88849960e-02],
        [ 2.71792969e+02, -2.66313553e-02, -1.88849960e-02],
        [ 2.79664062e+02,  1.86368942e-01, -1.98849961e-02]],

       [[-3.52539062e+02, -5.69753647e-02, -2.33549904e-02],
        [-3.47566406e+02, -1.47975445e-01,  3.56450118e-02],
        [-3.45542969e+02,  1.25024796e-01,  2.64500827e-03],
        ...,
        [ 2.91656250e+02,  5.80248833e-02, -1.53549910e-02],
        [ 2.95492188e+02, -2.59752274e-02, -1.63549911e-02],
        [ 2.96511719e+02, -5.59754372e-02, -3.35499272e-03]],

       [[-4.92183594e+02, -7.20443726e-02,  1.60599947e-02],
        [-4.89222656e+02, -8.20441246e-02, -1.94000453e-03],
        [-4.88269531e+02,  2.99558640e-02, -9.94000584e-03],
 

In [ ]:
# Attention mask is 1 when masked. 
# Random mask is 1 for masked observations selected to be randomized
# then,
att_mask    = tf.cast(lc_info_batches['att_mask'], tf.bool)
att_mask

<tf.Tensor: shape=(14, 200), dtype=bool, numpy=
array([[False, False, False, ..., False,  True,  True],
       [ True,  True, False, ..., False, False,  True],
       [ True, False, False, ...,  True, False, False],
       ...,
       [ True,  True,  True, ..., False,  True, False],
       [ True, False, False, ..., False, False, False],
       [ True, False,  True, ..., False,  True, False]])>

In [ ]:
random_mask = tf.cast(random_mask, tf.bool)
random_mask

<tf.Tensor: shape=(14, 200), dtype=bool, numpy=
array([[False, False, False, ..., False,  True, False],
       [ True, False, False, ..., False, False, False],
       [False, False, False, ...,  True, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False,  True, ..., False,  True, False]])>

In [ ]:
same_mask   = tf.cast(same_mask, tf.bool)
same_mask

<tf.Tensor: shape=(14, 200), dtype=bool, numpy=
array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False,  True, ..., False,  True, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])>

In [ ]:
att_mask

<tf.Tensor: shape=(14, 200), dtype=bool, numpy=
array([[False, False, False, ..., False,  True,  True],
       [ True,  True, False, ..., False, False,  True],
       [ True, False, False, ...,  True, False, False],
       ...,
       [ True,  True,  True, ..., False,  True, False],
       [ True, False, False, ..., False, False, False],
       [ True, False,  True, ..., False,  True, False]])>

In [ ]:
random_mask

<tf.Tensor: shape=(14, 200), dtype=bool, numpy=
array([[False, False, False, ..., False,  True, False],
       [ True, False, False, ..., False, False, False],
       [False, False, False, ...,  True, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False,  True, ..., False,  True, False]])>

In [ ]:
att_mask = tf.math.logical_xor(att_mask, random_mask)
att_mask

<tf.Tensor: shape=(14, 200), dtype=bool, numpy=
array([[False, False, False, ..., False, False,  True],
       [False,  True, False, ..., False, False,  True],
       [ True, False, False, ..., False, False, False],
       ...,
       [ True,  True,  True, ..., False,  True, False],
       [ True, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False]])>

In [ ]:
## Innecesariamente lo mismo
tf.cast(rest, tf.bool) == att_mask

<tf.Tensor: shape=(14, 200), dtype=bool, numpy=
array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])>

In [ ]:
att_mask = tf.math.logical_xor(att_mask, same_mask)
att_mask

<tf.Tensor: shape=(14, 200), dtype=bool, numpy=
array([[False, False, False, ..., False, False,  True],
       [False,  True, False, ..., False, False,  True],
       [ True, False, False, ..., False, False, False],
       ...,
       [ True,  True, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False]])>

In [ ]:
lc_info_batches.keys()

dict_keys(['input', 'original', 'lcid', 'length', 'mask', 'label', 'probed_mask', 'att_mask', 'input_pre_nsp'])

In [ ]:
np.sum(lc_info_batches['att_mask'][0])

80.0

In [ ]:
lc_info_batches['att_mask'] = tf.cast(att_mask, tf.float32)
lc_info_batches['att_mask'] 

<tf.Tensor: shape=(14, 200), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [ ]:
np.sum(lc_info_batches['att_mask'][0])

51.0

In [ ]:
for batch, lc_info_batches in enumerate(dataset_random_mask):
    print('Numero de batch {}'.format(batch))
    
    for snid, np_lc, mask_pad in zip(lc_info_batches['lcid'].numpy(), lc_info_batches['input'], lc_info_batches['mask']):
        #np_lc = np_lc[np_lc[:,0].argsort()]

        if snid == id_temporal:
            print('Encontramos la SNID')
            lc_temporal_padded = copy.deepcopy(np_lc)
            mask_padded = copy.deepcopy(mask_pad)
            lc_info_with_pad = copy.deepcopy(lc_info_batches)
    
        #print(lc_info['lcid'].numpy())
       # id_temporal = lc_info['lcid'].numpy()  
       # lc_temporal = copy.deepcopy(np_lc)

Numero de batch 0
Numero de batch 1
Numero de batch 2
Numero de batch 3
Numero de batch 4
Numero de batch 5
Numero de batch 6
Numero de batch 7
Numero de batch 8
Numero de batch 9
Numero de batch 10
Numero de batch 11
Numero de batch 12
Numero de batch 13
Numero de batch 14
Numero de batch 15
Numero de batch 16
Numero de batch 17
Numero de batch 18
Numero de batch 19
Numero de batch 20
Numero de batch 21
Numero de batch 22
Encontramos la SNID
Numero de batch 23
Numero de batch 24


In [ ]:
np.sum(lc_info_with_pad['att_mask'][13])

70.0

In [ ]:
np.sum(lc_info_with_pad['att_mask'][12])

51.0

In [ ]:
np.sum(lc_info_with_pad['probed_mask'][13])

76.0

In [ ]:
lc_info_with_pad['att_mask'][13]

<tf.Tensor: shape=(200,), dtype=float32, numpy=
array([0., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1.,
       0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
       1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)>

In [ ]:
lc_info_with_pad['probed_mask'][13]

<tf.Tensor: shape=(200,), dtype=float32, numpy=
array([1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 1.,
       1., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1.,
       0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0.,
       0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0.,
       0., 0., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1.,
       1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0.,
       0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1.,
       1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
       1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
       1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>

In [ ]:
lc_info_with_pad['att_mask'][13] == lc_info_with_pad['probed_mask'][13]

<tf.Tensor: shape=(200,), dtype=bool, numpy=
array([False,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True, False,  True, False,  True,  True,  True,  True, False,
        True, False,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True, False,
       False,  True,  True,  True, False,  True, False,  True,  True,
        True, False,  True, False, False,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  Tr

In [ ]:
200*0.2

40.0

In [ ]:
40*0.1

4.0

In [ ]:
36*0.1

3.6

In [ ]:
def format_input_no_nsp(input_dict, num_cls=None, test_mode=False):
    times = tf.slice(input_dict['input'], [0, 0, 0], [-1, -1, 1])  # input_dict['input'][0:-1, 0:-1, 0:1]
    magnitudes = tf.slice(input_dict['input'], [0, 0, 1], [-1, -1, 1]) # input_dict['input'][0:-1, 0:-1, 1:1]
    att_mask = tf.expand_dims(input_dict['att_mask'], axis=-1)

    inputs = {
        'magnitudes': magnitudes,
        'times': times,
        'att_mask': att_mask,
    }
    if test_mode:
        print('[INFO] TESTING MODE')
        inputs['original'] = input_dict['original']
        inputs['mask'] = input_dict['mask']

    if num_cls is not None:
        outputs = tf.one_hot(input_dict['label'], num_cls)

    else:
        outputs = {
            'magnitudes': tf.slice(input_dict['original'], [0, 0, 1], [-1, -1, 1]),
            'probed_mask': tf.expand_dims(input_dict['probed_mask'], -1),
        }

    return inputs, outputs

dataset_input = dataset_random_mask.map(lambda x: format_input_no_nsp(x, num_cls=num_cls, test_mode=test_mode))

In [ ]:
for batch, lc_info_batches in enumerate(dataset_input):
    print('Numero de batch {}'.format(batch))
    print(lc_info_batches)

    break
    
    for snid, np_lc in zip(lc_info_batches['lcid'].numpy(), lc_info_batches['input']):
        #np_lc = np_lc[np_lc[:,0].argsort()]

        if snid == id_temporal:
            print('Encontramos la SNID')
            lc_temporal_padded = copy.deepcopy(np_lc)

Numero de batch 0
({'magnitudes': <tf.Tensor: shape=(16, 200, 1), dtype=float32, numpy=
array([[[-0.01758671],
        [ 0.00441313],
        [-0.02958679],
        ...,
        [-0.06358671],
        [-0.0045867 ],
        [ 0.22541332]],

       [[ 0.0615387 ],
        [ 0.16353893],
        [ 0.00153875],
        ...,
        [-0.09246111],
        [ 0.17153883],
        [-0.19946098]],

       [[-0.25677538],
        [ 0.10322428],
        [ 0.05122423],
        ...,
        [ 0.12222433],
        [-0.13177538],
        [ 0.09822464]],

       ...,

       [[-0.1143198 ],
        [ 0.04568005],
        [-0.14432001],
        ...,
        [ 0.09768009],
        [ 0.23867989],
        [-0.04631996]],

       [[ 0.15093136],
        [ 0.19893122],
        [-0.02106857],
        ...,
        [ 0.05293131],
        [-0.10606861],
        [-0.1560688 ]],

       [[-0.05855036],
        [-0.13255024],
        [ 0.16544962],
        ...,
        [ 0.11444998],
        [-0.35855007],
      

In [ ]:
dict_input_lc = lc_info_batches[0]
dict_output_lc = lc_info_batches[1]

print(dict_input_lc.keys())
print(dict_output_lc.keys())

dict_keys(['magnitudes', 'times', 'att_mask'])
dict_keys(['magnitudes', 'probed_mask'])


In [ ]:
dict_input_lc['att_mask'][0] == dict_input_lc['att_mask'][1]

<tf.Tensor: shape=(200, 1), dtype=bool, numpy=
array([[False],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [False],
       [ True],
       [ True],
       [False],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [False],
       [False],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [F

In [ ]:
dict_input_lc['att_mask'][0]

<tf.Tensor: shape=(200, 1), dtype=float32, numpy=
array([[1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
 

In [ ]:
dict_input_lc['att_mask']

<tf.Tensor: shape=(16, 200, 1), dtype=float32, numpy=
array([[[1.],
        [1.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [1.]],

       [[0.],
        [1.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       ...,

       [[0.],
        [1.],
        [1.],
        ...,
        [0.],
        [0.],
        [1.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [1.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]], dtype=float32)>

In [ ]:
tf.expand_dims(lc_info_batches['att_mask'], axis=-1)

<tf.Tensor: shape=(14, 200, 1), dtype=float32, numpy=
array([[[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [1.],
        [0.]],

       [[1.],
        [0.],
        [0.],
        ...,
        [1.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       ...,

       [[0.],
        [1.],
        [0.],
        ...,
        [1.],
        [0.],
        [0.]],

       [[1.],
        [0.],
        [0.],
        ...,
        [1.],
        [1.],
        [0.]],

       [[1.],
        [0.],
        [0.],
        ...,
        [1.],
        [0.],
        [0.]]], dtype=float32)>

In [ ]:
SHUFFLE_BUFFER = 10000
dataset_input = dataset_input.shuffle(SHUFFLE_BUFFER)
dataset_input

<_ShuffleDataset element_spec=({'magnitudes': TensorSpec(shape=(None, 200, 1), dtype=tf.float32, name=None), 'times': TensorSpec(shape=(None, 200, 1), dtype=tf.float32, name=None), 'att_mask': TensorSpec(shape=(None, 200, 1), dtype=tf.float32, name=None)}, {'magnitudes': TensorSpec(shape=(None, 200, 1), dtype=tf.float32, name=None), 'probed_mask': TensorSpec(shape=(None, 200, 1), dtype=tf.float32, name=None)})>

In [ ]:
dataset_input = dataset_input.prefetch(2)
dataset_input

<_PrefetchDataset element_spec=({'magnitudes': TensorSpec(shape=(None, 200, 1), dtype=tf.float32, name=None), 'times': TensorSpec(shape=(None, 200, 1), dtype=tf.float32, name=None), 'att_mask': TensorSpec(shape=(None, 200, 1), dtype=tf.float32, name=None)}, {'magnitudes': TensorSpec(shape=(None, 200, 1), dtype=tf.float32, name=None), 'probed_mask': TensorSpec(shape=(None, 200, 1), dtype=tf.float32, name=None)})>

In [ ]:
for batch, lc_info_batches in enumerate(dataset_input):
    print('Numero de batch {}'.format(batch))
    print(lc_info_batches)
    
    for snid, np_lc in zip(lc_info_batches['lcid'].numpy(), lc_info_batches['input']):
        #np_lc = np_lc[np_lc[:,0].argsort()]

        if snid == id_temporal:
            print('Encontramos la SNID')
            lc_temporal_padded = copy.deepcopy(np_lc)

Numero de batch 0
({'magnitudes': <tf.Tensor: shape=(16, 200, 1), dtype=float32, numpy=
array([[[-0.03971672],
        [ 0.08628321],
        [-0.37771654],
        ...,
        [ 0.23528337],
        [ 0.04128361],
        [ 0.20028353]],

       [[ 0.0513587 ],
        [-0.01664162],
        [-0.05264139],
        ...,
        [ 0.15635872],
        [-0.1696415 ],
        [-0.07564116]],

       [[-0.27462006],
        [ 0.2023797 ],
        [ 0.20337963],
        ...,
        [ 0.13537979],
        [ 0.14737988],
        [ 0.13238001]],

       ...,

       [[-0.1050725 ],
        [ 0.10392761],
        [-0.11707258],
        ...,
        [ 0.10592747],
        [-0.04107285],
        [-0.06407261]],

       [[ 0.20767021],
        [ 0.11067057],
        [ 0.06767035],
        ...,
        [-0.06532955],
        [ 0.1866703 ],
        [-0.12732935]],

       [[-0.06105042],
        [ 0.03894997],
        [-0.20905018],
        ...,
        [-0.07805014],
        [-0.04305029],
      

TypeError: tuple indices must be integers or slices, not str

In [ ]:
dict_input_lc = lc_info_batches[0]
dict_output_lc = lc_info_batches[1]

print(dict_input_lc.keys())
print(dict_output_lc.keys())

dict_keys(['magnitudes', 'times', 'att_mask'])
dict_keys(['magnitudes', 'probed_mask'])


In [ ]:
dict_input_lc['magnitudes']

<tf.Tensor: shape=(16, 200, 1), dtype=float32, numpy=
array([[[-0.03971672],
        [ 0.08628321],
        [-0.37771654],
        ...,
        [ 0.23528337],
        [ 0.04128361],
        [ 0.20028353]],

       [[ 0.0513587 ],
        [-0.01664162],
        [-0.05264139],
        ...,
        [ 0.15635872],
        [-0.1696415 ],
        [-0.07564116]],

       [[-0.27462006],
        [ 0.2023797 ],
        [ 0.20337963],
        ...,
        [ 0.13537979],
        [ 0.14737988],
        [ 0.13238001]],

       ...,

       [[-0.1050725 ],
        [ 0.10392761],
        [-0.11707258],
        ...,
        [ 0.10592747],
        [-0.04107285],
        [-0.06407261]],

       [[ 0.20767021],
        [ 0.11067057],
        [ 0.06767035],
        ...,
        [-0.06532955],
        [ 0.1866703 ],
        [-0.12732935]],

       [[-0.06105042],
        [ 0.03894997],
        [-0.20905018],
        ...,
        [-0.07805014],
        [-0.04305029],
        [-0.14605045]]], dtype=float32)>

In [ ]:
dict_input_lc['att_mask']

<tf.Tensor: shape=(16, 200, 1), dtype=float32, numpy=
array([[[0.],
        [1.],
        [0.],
        ...,
        [1.],
        [1.],
        [0.]],

       [[1.],
        [0.],
        [0.],
        ...,
        [1.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [1.],
        [1.]],

       ...,

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [1.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [1.],
        ...,
        [0.],
        [0.],
        [0.]]], dtype=float32)>

In [ ]:
list_data_lcs = []

for lc_info in dataset_1:
    np_lc = lc_info['input'].numpy()
    np_lc = np_lc[np_lc[:,0].argsort()]

    for snid, lc_data, label in zip(lc_info['lcid'].numpy(), np_lc, lc_info['label'].numpy()):
        data = {'lcid': snid, 
                'lc_data': [lc_data],
                'label': label}  

        list_data_lcs.append(pd.DataFrame(data))

list_data_lcs = pd.concat(list_data_lcs)

In [ ]:
list_data_lcs

,lcid,lc_data,label
0,"b""b'3.7081.943'""","[[[49461.51, -4.964, 0.059], [49462.477, -4.86...",5
0,"b""b'3.7081.943'""","[[[49461.51, -4.964, 0.059], [49462.477, -4.86...",5
0,"b""b'3.7081.943'""","[[[49461.51, -4.964, 0.059], [49462.477, -4.86...",5
0,"b""b'3.7081.943'""","[[[49461.51, -4.964, 0.059], [49462.477, -4.86...",5
0,"b""b'10.3430.1344'""","[[[49461.51, -4.964, 0.059], [49462.477, -4.86...",5
...,...,...,...
0,"b""b'6.7054.88'""","[[[49819.547, -5.919, 0.035], [49824.52, -6.05...",2
0,"b""b'6.7054.88'""","[[[49819.547, -5.919, 0.035], [49824.52, -6.05...",2
0,"b""b'6.7054.88'""","[[[49819.547, -5.919, 0.035], [49824.52, -6.05...",2
0,"b""b'6.7054.88'""","[[[49819.547, -5.919, 0.035], [49824.52, -6.05...",2


In [ ]:
list_data_lcs[list_data_lcs.lcid == b"b'9.5486.943'"].lc_data.iloc[0][-1]

array([[ 4.9018617e+04, -4.3959999e+00,  1.0000000e-01],
       [ 4.9059512e+04, -4.9980001e+00,  8.9000002e-02],
       [ 4.9060492e+04, -5.0289998e+00,  8.6999997e-02],
       [ 4.9061480e+04, -4.8569999e+00,  7.2999999e-02],
       [ 4.9065648e+04, -5.0110002e+00,  9.6000001e-02],
       [ 4.9068523e+04, -4.9050002e+00,  7.9000004e-02],
       [ 4.9077484e+04, -5.2030001e+00,  9.0000004e-02],
       [ 4.9094438e+04, -4.7940001e+00,  9.0000004e-02],
       [ 4.9095438e+04, -4.9159999e+00,  9.4999999e-02],
       [ 4.9096434e+04, -4.5700002e+00,  9.0000004e-02],
       [ 4.9103434e+04, -5.1259999e+00,  7.4000001e-02],
       [ 4.9120418e+04, -5.1310000e+00,  7.8000002e-02],
       [ 4.9145398e+04, -6.1059999e+00, -4.5600000e+02],
       [ 4.9283766e+04, -5.2230000e+00,  7.5999998e-02],
       [ 4.9447633e+04, -4.8439999e+00,  9.7000003e-02],
       [ 4.9458527e+04, -4.7319999e+00,  9.4999999e-02],
       [ 4.9465496e+04, -4.7989998e+00,  1.0000000e-01],
       [ 4.9518430e+04, -2.6750

In [ ]:
dataset_windows[dataset_windows.lcid == b"b'9.5486.943'"].lc_data.iloc[0]

array([[ 4.9018617e+04, -4.3959999e+00,  1.0000000e-01],
       [ 4.9059512e+04, -4.9980001e+00,  8.9000002e-02],
       [ 4.9060492e+04, -5.0289998e+00,  8.6999997e-02],
       [ 4.9061480e+04, -4.8569999e+00,  7.2999999e-02],
       [ 4.9065648e+04, -5.0110002e+00,  9.6000001e-02],
       [ 4.9068523e+04, -4.9050002e+00,  7.9000004e-02],
       [ 4.9077484e+04, -5.2030001e+00,  9.0000004e-02],
       [ 4.9094438e+04, -4.7940001e+00,  9.0000004e-02],
       [ 4.9095438e+04, -4.9159999e+00,  9.4999999e-02],
       [ 4.9096434e+04, -4.5700002e+00,  9.0000004e-02],
       [ 4.9103434e+04, -5.1259999e+00,  7.4000001e-02],
       [ 4.9120418e+04, -5.1310000e+00,  7.8000002e-02],
       [ 4.9145398e+04, -6.1059999e+00, -4.5600000e+02],
       [ 4.9283766e+04, -5.2230000e+00,  7.5999998e-02],
       [ 4.9447633e+04, -4.8439999e+00,  9.7000003e-02],
       [ 4.9458527e+04, -4.7319999e+00,  9.4999999e-02],
       [ 4.9465496e+04, -4.7989998e+00,  1.0000000e-01],
       [ 4.9518430e+04, -2.6750

In [ ]:
import six

def assert_rank(tensor, expected_rank, name=None):
  """Raises an exception if the tensor rank is not of the expected rank.

  Args:
    tensor: A tf.Tensor to check the rank of.
    expected_rank: Python integer or list of integers, expected rank.
    name: Optional name of the tensor for the error message.

  Raises:
    ValueError: If the expected shape doesn't match the actual shape.
  """
  if name is None:
    name = tensor.name

  expected_rank_dict = {}
  if isinstance(expected_rank, six.integer_types):
    expected_rank_dict[expected_rank] = True
  else:
    for x in expected_rank:
      expected_rank_dict[x] = True

  actual_rank = tensor.shape.ndims
  if actual_rank not in expected_rank_dict:
    scope_name = tf.get_variable_scope().name
    raise ValueError(
        "For the tensor `%s` in scope `%s`, the actual rank "
        "`%d` (shape = %s) is not equal to the expected rank `%s`" %
        (name, scope_name, actual_rank, str(tensor.shape), str(expected_rank)))
  

def get_shape_list(tensor, expected_rank=None, name=None):
  """Returns a list of the shape of tensor, preferring static dimensions.

  Args:
    tensor: A tf.Tensor object to find the shape of.
    expected_rank: (optional) int. The expected rank of `tensor`. If this is
      specified and the `tensor` has a different rank, and exception will be
      thrown.
    name: Optional name of the tensor for the error message.

  Returns:
    A list of dimensions of the shape of tensor. All static dimensions will
    be returned as python integers, and dynamic dimensions will be returned
    as tf.Tensor scalars.
  """
  if name is None:
    name = tensor.name

  if expected_rank is not None:
    assert_rank(tensor, expected_rank, name)

  shape = tensor.shape.as_list()

  non_static_indexes = []
  for (index, dim) in enumerate(shape):
    if dim is None:
      non_static_indexes.append(index)

  if not non_static_indexes:
    return shape

  dyn_shape = tf.shape(tensor)
  for index in non_static_indexes:
    shape[index] = dyn_shape[index]
  return shape


def create_attention_mask_from_input_mask(from_tensor, to_mask):
    """Create 3D attention mask from a 2D tensor mask.

    Args:
    from_tensor: 2D or 3D Tensor of shape [batch_size, from_seq_length, ...].
    to_mask: int32 Tensor of shape [batch_size, to_seq_length].

    Returns:
    float Tensor of shape [batch_size, from_seq_length, to_seq_length].
    """
    from_shape = get_shape_list(from_tensor, expected_rank=[2, 3])
    batch_size = from_shape[0]
    from_seq_length = from_shape[1]

    to_shape = get_shape_list(to_mask, expected_rank=2)
    to_seq_length = to_shape[1]

    to_mask = tf.cast(
        tf.reshape(to_mask, [batch_size, 1, to_seq_length]), tf.float32)

    # We don't assume that `from_tensor` is a mask (although it could be). We
    # don't actually care if we attend *from* padding tokens (only *to* padding)
    # tokens so we create a tensor of all ones.
    #
    # `broadcast_ones` = [batch_size, from_seq_length, 1]
    broadcast_ones = tf.ones(
        shape=[batch_size, from_seq_length, 1], dtype=tf.float32)

    # Here we broadcast along two dimensions to create the mask.
    mask = broadcast_ones * to_mask

    return mask

In [ ]:
dict_input_lc['att_mask']

<tf.Tensor: shape=(16, 200, 1), dtype=float32, numpy=
array([[[1.],
        [1.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [1.]],

       [[0.],
        [1.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       ...,

       [[0.],
        [1.],
        [1.],
        ...,
        [0.],
        [0.],
        [1.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [1.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]], dtype=float32)>